**Import Libraries and read the dataset**

In [141]:
import numpy as np
import pandas as pd
from sklearn import metrics
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import tensorflow as tf
# Using Tensorflow Keras instead of the original Keras

from tensorflow.keras.datasets import mnist
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score, precision_recall_curve, auc
import matplotlib.pyplot as plt
from tensorflow.keras import optimizers
from google.colab import drive

In [142]:
drive.mount('/content/drive/')
project_path = '/content/drive/My Drive/Colab Notebooks/'
dataset_file = project_path + 'bank.csv'

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [143]:
# reading the CSV file into pandas dataframe
df = pd.read_csv(dataset_file)  

pd.isnull(df).count()

RowNumber          10000
CustomerId         10000
Surname            10000
CreditScore        10000
Geography          10000
Gender             10000
Age                10000
Tenure             10000
Balance            10000
NumOfProducts      10000
HasCrCard          10000
IsActiveMember     10000
EstimatedSalary    10000
Exited             10000
dtype: int64

In [144]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


**Perform basic data exporation**

In [145]:
df.shape

(10000, 14)

In [146]:
df.describe()

,RowNumber,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,10000.00000,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000
mean,5000.50000,1.569094e+07,650.528800,38.921800,5.012800,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203700
std,2886.89568,7.193619e+04,96.653299,10.487806,2.892174,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769
min,1.00000,1.556570e+07,350.000000,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000
25%,2500.75000,1.562853e+07,584.000000,32.000000,3.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000
50%,5000.50000,1.569074e+07,652.000000,37.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000
75%,7500.25000,1.575323e+07,718.000000,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000
max,10000.00000,1.581569e+07,850.000000,92.000000,10.000000,250898.090000,4.000000,1.00000,1.000000,199992.480000,1.000000


In [147]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


**2. Drop the columns which are unique for all users like IDs (5 points)**

In [148]:
df  = df.drop("RowNumber" , axis=1)
df  = df.drop("CustomerId" , axis=1)
df  = df.drop("Surname" , axis=1)

In [149]:
df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [150]:
df.isna().sum()

CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

**3. Distinguish the feature and target set (5 points)**
Here, our target variable is 'Exited' and rest are the feature set.

In [151]:
X_data = df.iloc[:, 1:-1]
y_data = df.iloc[:, -1]

In [152]:
X_data.shape


(10000, 9)

In [153]:
y_data.shape

(10000,)

In [154]:
y_data.head()

0    1
1    0
2    1
3    0
4    0
Name: Exited, dtype: int64

In [155]:
y_data.value_counts()

0    7963
1    2037
Name: Exited, dtype: int64

In [156]:
X_data.head()

,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,France,Female,42,2,0.00,1,1,1,101348.88
1,Spain,Female,41,1,83807.86,1,0,1,112542.58
2,France,Female,42,8,159660.80,3,1,0,113931.57
3,France,Female,39,1,0.00,2,0,0,93826.63
4,Spain,Female,43,2,125510.82,1,1,1,79084.10


*Using one hot encoding and replacing object fields with numerical values to represent the same.*

In [157]:
replaceStruct = {
                "Geography": {"France": 1, "Germany": 2 ,"Spain": 3},
                "Gender": {"Female": 0, "Male": 1}, 
                }
oneHotCols=["Geography","Gender"]

In [158]:
X_data=X_data.replace(replaceStruct)
X_data.head()

,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,1,0,42,2,0.00,1,1,1,101348.88
1,3,0,41,1,83807.86,1,0,1,112542.58
2,1,0,42,8,159660.80,3,1,0,113931.57
3,1,0,39,1,0.00,2,0,0,93826.63
4,3,0,43,2,125510.82,1,1,1,79084.10


In [159]:
X_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Geography        10000 non-null  int64  
 1   Gender           10000 non-null  int64  
 2   Age              10000 non-null  int64  
 3   Tenure           10000 non-null  int64  
 4   Balance          10000 non-null  float64
 5   NumOfProducts    10000 non-null  int64  
 6   HasCrCard        10000 non-null  int64  
 7   IsActiveMember   10000 non-null  int64  
 8   EstimatedSalary  10000 non-null  float64
dtypes: float64(2), int64(7)
memory usage: 703.2 KB


In [160]:

X_data.head()

,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,1,0,42,2,0.00,1,1,1,101348.88
1,3,0,41,1,83807.86,1,0,1,112542.58
2,1,0,42,8,159660.80,3,1,0,113931.57
3,1,0,39,1,0.00,2,0,0,93826.63
4,3,0,43,2,125510.82,1,1,1,79084.10


**4. Divide the data set into training and test sets (5 points)**

In [161]:
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size = 0.2, random_state = 7)

**5. Normalize the train and test data (10 points)**

*Normalizing*

In [162]:
X_train = preprocessing.normalize(X_train)

In [163]:
X_test = preprocessing.normalize(X_test)

In [164]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(8000, 9)
(2000, 9)
(8000,)
(2000,)


*Scaling*

In [165]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

6. Initialize & build the model. Identify the points of improvement and implement the same the same.(20)

In [166]:
from tensorflow import keras
import keras
from keras.layers import Dense

In [167]:
model = Sequential()

In [168]:
model.add(Dense(6, input_shape = (9,), activation = 'relu'))   #from input layer

In [169]:
model.add(Dense(units= 6, activation = 'relu', kernel_initializer="uniform"))   #adding hidden layers

In [170]:
model.add(Dense(units = 1, activation = 'sigmoid', kernel_initializer="uniform"))  #to output layer

In [171]:
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [172]:
model.fit(X_train, y_train, batch_size = 10, epochs = 100)

Epoch 1/100
800/800 [==============================] - 1s 896us/step - loss: 0.5518 - accuracy: 0.7962
Epoch 2/100
800/800 [==============================] - 1s 894us/step - loss: 0.4988 - accuracy: 0.7968
Epoch 3/100
800/800 [==============================] - 1s 866us/step - loss: 0.4984 - accuracy: 0.7968
Epoch 4/100
800/800 [==============================] - 1s 909us/step - loss: 0.4977 - accuracy: 0.7968
Epoch 5/100
800/800 [==============================] - 1s 957us/step - loss: 0.4974 - accuracy: 0.7968
Epoch 6/100
800/800 [==============================] - 1s 986us/step - loss: 0.4968 - accuracy: 0.7968
Epoch 7/100
800/800 [==============================] - 1s 876us/step - loss: 0.4967 - accuracy: 0.7968
Epoch 8/100
800/800 [==============================] - 1s 898us/step - loss: 0.4961 - accuracy: 0.7968
Epoch 9/100
800/800 [==============================] - 1s 916us/step - loss: 0.4956 - accuracy: 0.7968
Epoch 10/100
800/800 [==============================] - 1s 915us/step - l

In [173]:
model.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_29 (Dense)             (None, 6)                 60        
_________________________________________________________________
dense_30 (Dense)             (None, 6)                 42        
_________________________________________________________________
dense_31 (Dense)             (None, 1)                 7         
Total params: 109
Trainable params: 109
Non-trainable params: 0
_________________________________________________________________


In [174]:
loss, acc = model.evaluate(X_test, y_test, verbose=0)
print('Accuracy: %.3f'  % acc)
print('Loss: %.3f' % loss)

Accuracy: 0.807
Loss: 0.446


In [193]:
   model.fit(X_train, y_train, batch_size = 10, epochs = 190)

Epoch 1/190
800/800 [==============================] - 1s 885us/step - loss: 0.4178 - accuracy: 0.8142
Epoch 2/190
800/800 [==============================] - 1s 880us/step - loss: 0.4179 - accuracy: 0.8136
Epoch 3/190
800/800 [==============================] - 1s 895us/step - loss: 0.4178 - accuracy: 0.8148
Epoch 4/190
800/800 [==============================] - 1s 865us/step - loss: 0.4181 - accuracy: 0.8158
Epoch 5/190
800/800 [==============================] - 1s 914us/step - loss: 0.4178 - accuracy: 0.8154
Epoch 6/190
800/800 [==============================] - 1s 916us/step - loss: 0.4182 - accuracy: 0.8149
Epoch 7/190
800/800 [==============================] - 1s 870us/step - loss: 0.4184 - accuracy: 0.8160
Epoch 8/190
800/800 [==============================] - 1s 893us/step - loss: 0.4177 - accuracy: 0.8158
Epoch 9/190
800/800 [==============================] - 1s 880us/step - loss: 0.4178 - accuracy: 0.8142
Epoch 10/190
800/800 [==============================] - 1s 875us/step - l

In [198]:
loss, acc = model.evaluate(X_test, y_test, verbose=0)
print('Accuracy: %.3f'  % acc)
print('Loss: %.3f' % loss)

#we get better result with epochs=190

Accuracy: 0.811
Loss: 0.428


**7. Predict the results using 0.5 as a threshold (10 points)**

In [195]:
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5) #if y_pred is larger than 0.5 it returns true(1) else false(2)
print(y_pred)

[[False]
 [False]
 [False]
 ...
 [False]
 [False]
 [False]]


**8. Print the Accuracy score and confusion matrix (5 points)**

In [196]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[1527   62]
 [ 317   94]]


In [197]:
cr=metrics.classification_report(y_test,y_pred)
print(cr)

              precision    recall  f1-score   support

           0       0.83      0.96      0.89      1589
           1       0.60      0.23      0.33       411

    accuracy                           0.81      2000
   macro avg       0.72      0.59      0.61      2000
weighted avg       0.78      0.81      0.77      2000



* From the above condusion matrix, our True positive rates and true negative rates are high which is good. False negative case of 317 is not a problem in our context as the customers are anyways staying and not churning. False positive rate is low which is what we want.
Accuracy rates are also nearly 81% which is really good in our case.
Hence our model seems to be working pretty well!! 